In [1]:
from task import CreateDataLabel,MapAtomNode,node_accuracy
from schnet import SchNetModel
#from proteinworkshop.models.graph_encoders.schnet import SchNetModel
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import wandb
import seaborn as sns

[12/11/23 19:09:36] WARNING  To use the Graphein submodule                                         ]8;id=998916;file:///zhome/6e/5/146981/.local/lib/python3.9/site-packages/graphein/protein/features/sequence/embeddings.py\embeddings.py]8;;\:]8;id=859622;file:///zhome/6e/5/146981/.local/lib/python3.9/site-packages/graphein/protein/features/sequence/embeddings.py#44\44]8;;\
                             graphein.protein.features.sequence.embeddings, you need to install:                   
                             biovec                                                                                
                             biovec cannot be installed via conda                                                  
                             Alternatively, you can install graphein with the extras:                              
                                                                                                                   
                             pip install graphein[extras]                                                          

[12/11/23 19:09:39] WARNING  To use the Graphein submodule graphein.protein.visualisation, you  ]8;id=514819;file:///zhome/6e/5/146981/.local/lib/python3.9/site-packages/graphein/protein/visualisation.py\visualisation.py]8;;\:]8;id=99621;file:///zhome/6e/5/146981/.local/lib/python3.9/site-packages/graphein/protein/visualisation.py#35\35]8;;\
                             need to install: pytorch3d                                                            
                             To do so, use the following command: conda install -c pytorch3d                       
                             pytorch3d                                                                             

                    WARNING  To use the Graphein submodule graphein.protein.meshes, you need to        ]8;id=332507;file:///zhome/6e/5/146981/.local/lib/python3.9/site-packages/graphein/protein/meshes.py\meshes.py]8;;\:]8;id=211772;file:///zhome/6e/5/146981/.local/lib/python3.9/site-packages/graphein/protein/meshes.py#29\29]8;;\
                             install: pytorch3d                                                                    
                             To do so, use the following command: conda install -c pytorch3d pytorch3d             

In [2]:
##['train', 'val', 'test_SP_TM', 'test_TM', 'test_BETA']
batch_size = 1
raw_data_name = "DeepTMHMM.3line"
path ='/work3/s194408/Project/'
processor = CreateDataLabel(path,batch_size =batch_size,raw_data_name=raw_data_name)
# processor.initialization()# split and download trian/val/test just once

train_data,train_lable, train_batchname, train_max_len,train_dismatch_index_pred,train_dismatch_index_type,train_real_node_label,df_train = processor.datalabelgenerator('train')

val_data,val_lable, val_batchname, val_max_len,val_dismatch_index_pred,val_dismatch_index_type,val_real_node_label,df_val = processor.datalabelgenerator('val')

test_SP_TM_data,test_SP_TM_lable, test_SP_TM_batchname, test_SP_TM_max_len,test_SP_TM_dismatch_index_pred,test_SP_TM_dismatch_index_type,test_SP_TM_real_node_label,df_test_SP_TM = processor.datalabelgenerator('test_SP_TM')

test_TM_data,test_TM_lable, test_TM_batchname, test_TM_max_len,test_TM_dismatch_index_pred,test_TM_dismatch_index_type,test_TM_real_node_label,df_test_TM = processor.datalabelgenerator('test_TM')

test_BETA_data,test_BETA_lable, test_BETA_batchname, test_BETA_max_len,test_BETA_dismatch_index_pred,test_BETA_dismatch_index_type,test_BETA_real_node_label,df_test_BETA = processor.datalabelgenerator('test_BETA')

[12/11/23 10:10:28] INFO     Amending traindata                                                          ]8;id=48946;file:///work3/s194408/Project/schnet_majorvoting/task.py\task.py]8;;\:]8;id=363645;file:///work3/s194408/Project/schnet_majorvoting/task.py#78\78]8;;\

[12/11/23 10:10:29] INFO     Found 2381 examples in train                                         ]8;id=866064;file:///work3/s194408/Project/schnet_majorvoting/data_utils.py\data_utils.py]8;;\:]8;id=890192;file:///work3/s194408/Project/schnet_majorvoting/data_utils.py#307\307]8;;\

Finding dismatch and processing ...: 100%|█| 2381/2381 [00:11<00:00, 198.72it/s]


[12/11/23 10:15:37] INFO     Have finshed finding dismatch and processing, after processing not   ]8;id=860423;file:///work3/s194408/Project/schnet_majorvoting/data_utils.py\data_utils.py]8;;\:]8;id=661208;file:///work3/s194408/Project/schnet_majorvoting/data_utils.py#143\143]8;;\
                             founded dismatch                                                                      

                    INFO     Processing train data                                                       ]8;id=416466;file:///work3/s194408/Project/schnet_majorvoting/task.py\task.py]8;;\:]8;id=148536;file:///work3/s194408/Project/schnet_majorvoting/task.py#82\82]8;;\

...

                    INFO     Processing val data                                                         ]8;id=807031;file:///work3/s194408/Project/schnet_majorvoting/task.py\task.py]8;;\:]8;id=202963;file:///work3/s194408/Project/schnet_majorvoting/task.py#82\82]8;;\

Structure Analysis Using Geometric: 100%|█████| 596/596 [00:18<00:00, 32.12it/s]


[12/11/23 10:23:29] INFO     Processing val labels                                                       ]8;id=655951;file:///work3/s194408/Project/schnet_majorvoting/task.py\task.py]8;;\:]8;id=224358;file:///work3/s194408/Project/schnet_majorvoting/task.py#87\87]8;;\

                    INFO     Found 596 examples in val                                            ]8;id=556068;file:///work3/s194408/Project/schnet_majorvoting/data_utils.py\data_utils.py]8;;\:]8;id=945323;file:///work3/s194408/Project/schnet_majorvoting/data_utils.py#307\307]8;;\

Processing val atom label: 100%|████████████| 596/596 [00:00<00:00, 1840.25it/s]


In [3]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="DL_tmp", #项目名称
    entity="transmembrane-topology", # 用户名
    group="major voting", # 对比实验分组
    name= "batchsize 1 with weight ", #实验的名字
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.001,
    "architecture": "schnet",
    "dataset": "protein 3D structures ",
    "epochs":150,
    'batch_size':8,
    'hidden_channels' :256,
    'weight_decay': 1e-4
    }
)
sns.set_style("whitegrid")

wandb: Currently logged in as: s194408 (transmembrane-topology). Use `wandb login --relogin` to force relogin


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f8927005dc0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f8927005670, execution_count=3 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f892703f6d0, raw_cell="# start a new wandb run to track this script
wandb.." store_history=True silent=False shell_futures=True cell_id=None> result=None>,),kwargs {}:


TypeError: _pause_backend() takes 1 positional argument but 2 were given

In [2]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#max_len=max(train_max_len,val_max_len,test_SP_TM_max_len,test_TM_max_len,test_BETA_max_len)+1 #StaticEmbedding need max_len
# put model to GPU
#model = SchNetModel(hidden_channels=256, out_dim=6, max_len=30000).to(device)
#criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-4)



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#max_len=max(train_max_len,val_max_len,test_SP_TM_max_len,test_TM_max_len,test_BETA_max_len)+1 #StaticEmbedding need max_len
# put model to GPU
model = SchNetModel(hidden_channels=256, out_dim=6).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-4)

chk_path = '/work3/s194408/Project/schnet_weight/ca_bb/last.ckpt'
checkpoint  = torch.load(chk_path, map_location=torch.device('cpu'))

model.load_state_dict(checkpoint, strict=False)


TypeError: __init__() missing 1 required positional argument: 'max_len'

In [ ]:
# Freeze the weights

for param in model.parameters():
  param.requires_grad = False


In [6]:
total_epochs=150
draw_num = 1
global_step = 0


epoch_atom_level_accuracy_record_train = []
epoch_loss_record_train=[]
epoch_node_level_accuracy_record_train = []
epoch_atom_level_accuracy_record_val = []
epoch_loss_record_val = []
epoch_node_level_accuracy_record_val = []
for epoch in range(total_epochs):
    epoch_atom_level_accuracy_train = []
    epoch_loss_train=[]
    epoch_node_level_accuracy_train = []
    # train
    for i, data in enumerate(train_data):  
        global_step += 1 
        optimizer.zero_grad()  
        outputs = model(data.to(device))   # put batch data in GPU get logits
        prediction = outputs["node_embedding"]  
        real_label = torch.argmax(torch.tensor(train_lable[i]), dim=1).to(device) # put label in GPU
        loss = criterion(prediction, real_label)  # operate in the same device
        loss.backward()     
        optimizer.step()    

        #calulate atom-level accuracy and node-level accuracy
        _, predicted = torch.max(prediction, 1) 
        correct = (predicted == real_label).sum().item()
        total = real_label.size(0)
        atom_level_accuracy =  correct / total

        # below is operated under CPU node
        processor = MapAtomNode(predicted.cpu(),train_batchname[i],train_dismatch_index_pred,train_dismatch_index_type,df_train)

        train_predict_node_label = processor.map_atom_node() 
        node_level_accuracy = node_accuracy(train_predict_node_label,train_real_node_label[i])

        wandb.log({'train_loss_step':loss.item(), 'global_step':global_step})
        wandb.log({'train_atom_level_accuracy_step':atom_level_accuracy,  'global_step':global_step})
        wandb.log({'train_node_level_accuracy_step':node_level_accuracy, 'global_step':global_step})

        epoch_loss_train.append(loss.item())
        epoch_atom_level_accuracy_train.append(atom_level_accuracy)
        epoch_node_level_accuracy_train.append(node_level_accuracy)
        
    epoch_loss_record_train.append(np.mean(epoch_loss_train))
    epoch_atom_level_accuracy_record_train.append(np.mean(epoch_atom_level_accuracy_train))
    epoch_node_level_accuracy_record_train.append(np.mean(epoch_node_level_accuracy_train))

    wandb.log({'train_loss_epoch':np.mean(epoch_loss_train), 'global_step':global_step})
    wandb.log({'train_atom_level_accuracy_epoch':np.mean(epoch_atom_level_accuracy_train),  'global_step':global_step})
    wandb.log({'train_node_level_accuracy_epoch':np.mean(epoch_node_level_accuracy_train), 'global_step':global_step})
    
    # val
    model.eval()  
    with torch.no_grad():  

        epoch_atom_level_accuracy_val = []
        epoch_loss_val = []
        epoch_node_level_accuracy_val = []
        
        for i, data in enumerate(val_data):  
            outputs = model(data.to(device))
            prediction = outputs["node_embedding"]
            real_label = torch.argmax(torch.tensor(val_lable[i]), dim=1).to(device)
            loss = criterion(prediction, real_label)

            
            _, predicted = torch.max(prediction, 1)
            correct = (predicted == real_label).sum().item()
            total = real_label.size(0)
            atom_level_accuracy = correct / total

            processor = MapAtomNode(predicted.cpu(), val_batchname[i], val_dismatch_index_pred, val_dismatch_index_type, df_val)
            val_predict_node_label = processor.map_atom_node()
            node_level_accuracy = node_accuracy(val_predict_node_label, val_real_node_label[i])

            epoch_loss_val.append(loss.item())
            epoch_atom_level_accuracy_val.append(atom_level_accuracy)
            epoch_node_level_accuracy_val.append(node_level_accuracy)
            
            wandb.log({'val_loss_step':loss.item(), 'global_step':global_step})
            wandb.log({'val_atom_level_accuracy_step':atom_level_accuracy,  'global_step':global_step})
            wandb.log({'val_node_level_accuracy_step':node_level_accuracy, 'global_step':global_step})


            epoch_loss_val.append(loss.item())
            epoch_atom_level_accuracy_val.append(atom_level_accuracy)
            epoch_node_level_accuracy_val.append(node_level_accuracy)
            
        epoch_loss_record_val.append(np.mean(epoch_loss_val))
        epoch_atom_level_accuracy_record_val.append(np.mean(epoch_atom_level_accuracy_val))
        epoch_node_level_accuracy_record_val.append(np.mean(epoch_node_level_accuracy_val))

        wandb.log({'val_loss_epoch':np.mean(epoch_loss_val), 'global_step':global_step})
        wandb.log({'val_atom_level_accuracy_epoch':np.mean(epoch_atom_level_accuracy_val), 'global_step':global_step})
        wandb.log({'val_node_level_accuracy_epoch':np.mean(epoch_node_level_accuracy_val), 'global_step':global_step})

    if epoch % draw_num == 0:
        print(f"EPOCH:{epoch}:Train Loss:{np.mean(epoch_loss_train)} Train Atom Level Accuracy:{np.mean(epoch_atom_level_accuracy_train)} Train Node Level Accuracy:{np.mean(epoch_node_level_accuracy_train)}")
        print(f"EPOCH:{epoch}:Val Loss:{np.mean(epoch_loss_val)} Val Atom Level Accuracy:{np.mean(epoch_atom_level_accuracy_val)} Val Node Level Accuracy:{np.mean(epoch_node_level_accuracy_val)}")


wandb.finish()


print("Finished training.")

torch.save(model.state_dict(), '/work3/s194408/Project/result/test_model.pth')

   
        


        



RuntimeError: expected scalar type Long but found Float

In [10]:
# to load

chk_path = '/work3/s194408/Project/weight/ca_bb/last.ckpt'

checkpoint  = torch.load(chk_path, map_location=torch.device('cpu'))


model.load_state_dict(checkpoint['state_dict'], strict=False)


_IncompatibleKeys(missing_keys=['embedding.embedding.weight', 'distance_expansion.offset', 'interactions.0.mlp.0.weight', 'interactions.0.mlp.0.bias', 'interactions.0.mlp.2.weight', 'interactions.0.mlp.2.bias', 'interactions.0.conv.lin1.weight', 'interactions.0.conv.lin2.weight', 'interactions.0.conv.lin2.bias', 'interactions.0.conv.nn.0.weight', 'interactions.0.conv.nn.0.bias', 'interactions.0.conv.nn.2.weight', 'interactions.0.conv.nn.2.bias', 'interactions.0.lin.weight', 'interactions.0.lin.bias', 'interactions.1.mlp.0.weight', 'interactions.1.mlp.0.bias', 'interactions.1.mlp.2.weight', 'interactions.1.mlp.2.bias', 'interactions.1.conv.lin1.weight', 'interactions.1.conv.lin2.weight', 'interactions.1.conv.lin2.bias', 'interactions.1.conv.nn.0.weight', 'interactions.1.conv.nn.0.bias', 'interactions.1.conv.nn.2.weight', 'interactions.1.conv.nn.2.bias', 'interactions.1.lin.weight', 'interactions.1.lin.bias', 'interactions.2.mlp.0.weight', 'interactions.2.mlp.0.bias', 'interactions.2.mlp

In [ ]:
# Try load the weight without using the module

chk_path = '/work3/s194408/Project/weight/ca_bb/last.ckpt'

# original saved file with DataParallel
checkpoint = torch.load(chk_path, map_location=torch.device('cpu'))

# create new OrderedDict that does not contain `module.`
from collections import OrderedDict

new_state_dict = OrderedDict()
for k, v in checkpoint.items():
    name = key.replace("module.", "") # remove `module.`
    new_state_dict[name] = v

# load params
model.load_state_dict(new_state_dict, strict=False)